In [1]:
import json
from functools import lru_cache
from operator import itemgetter
from typing import List

import pandas as pd
import requests
from pydantic import BaseModel, confloat, AnyUrl,conint
from ratelimit import RateLimitException, limits
from tenacity import *
from yarl import URL

In [4]:
API_ENDPOINT = URL("https://api.coingecko.com/api/v3")

In [5]:
@retry(
    retry=retry_if_exception_type(requests.exceptions.RequestException),
    wait=wait_exponential(max=60),
    stop=stop_after_attempt(3),
)
@retry(
    retry=retry_if_exception_type(RateLimitException),
    wait=wait_fixed(60),
    stop=stop_after_delay(360),
)
@limits(calls=50, period=60)
def call_get(url: URL, params: dict) -> str:
    return requests.get(url.update_query(params)).text

In [5]:
class MarketDataInfo(BaseModel):
    id: str
    symbol: str
    current_price: float
    market_cap: int

In [6]:
def parse_result(result) -> MarketDataInfo:
    return MarketDataInfo(
        id=result["id"],
        symbol=result["symbol"],
        current_price=result["current_price"],
        market_cap=result["market_cap"],
        image=result["image"]
    )

In [6]:
# list_of_coins = pd.read_csv("/home/vlad/master_project/data/coin_list/list_of_coins.csv")['id'].to_list()

In [14]:
#save as raw json
with open('zxc_historical_prices.json', 'w') as f:
    json.dump(a, f)

In [10]:
def get_historical_data(id: str) -> List[str]:
    return json.loads(
            call_get(API_ENDPOINT / "coins" / id / "market_chart", {"vs_currency" : "usd", "days" : "max"})
        )

In [3]:
import logging
import boto3
from botocore.exceptions import ClientError


def create_bucket(bucket_name, session, region=None):
    """Create an S3 bucket in a specified region

    If a region is not specified, the bucket is created in the S3 default
    region (us-east-1).

    :param bucket_name: Bucket to create
    :param region: String region to create bucket in, e.g., 'us-west-2'
    :return: True if bucket created, else False
    """

    # Create bucket
    try:
        
        if region is None:
            s3_client = session.client('s3')
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client = session.client('s3', region_name=region)
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name,
                                    CreateBucketConfiguration=location)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [4]:
session = boto3.Session(profile_name='master')

In [5]:
create_bucket('kosmobiker-test', session)

True

In [6]:
import awswrangler as wr


In [8]:
bucket  = "kosmobiker-test"
path1 = f"s3://{bucket}/csv/file1.csv"

In [10]:
df1 = pd.DataFrame({
    "id": [1, 2],
    "name": ["foo", "boo"]
})
df1

,id,name
0,1,foo
1,2,boo
